# [프로젝트4] 시계열 데이터 EDA를 통한 이상치 탐지 (1)

---


## 프로젝트 목표
---
- 탐색적 데이터 분석을 통해 데이터의 이상치를 탐지합니다.
- STL 기법과 자기상관 그래프를 통해 데이터의 이상탐지를 진행합니다.


## 프로젝트 목차
---

1. **판다스로 csv 파일 불러오기:** 데이터 분석을 위한 python 라이브러리 pandas를 활용하여 데이터를 불러옵니다.

2. **데이터 기본 분석하기:** 기초적인 데이터 구조를 확인합니다.

3. **여러 개의 변수를 통합하여 이상 장비 확인하기:** 여러 개의 변수를 통합하여 이상 장비를 확인합니다.

4. **STL 기법을 활용한 이상치 탐지:** STL 기법을 활용하여 이상 장비를 확인합니다. 

5. **Autocorrelation/Partial-Autocorrelation plot(자기상관 그래프)을 통한 이상탐지:** 자기상관 그래프를 통해 이상탐지를 진행합니다. 

## 프로젝트 개요
---

데이터 Exploratory Data Analysis(EDA)를 통해 데이터의 분포 및 값을 검토함으로써 데이터가 표현하는 현상을 더 잘 이해하고, 데이터에 대한 잠재적인 문제를 발견(이상치)해보는 과정을 진행합니다. 

---

## 1. 데이터 불러오기
---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
plt.rcParams['axes.unicode_minus']=False

In [ ]:
df=pd.read_csv('df_preprocessed(inter)_1004_0.csv')

In [ ]:
df

In [ ]:
df.count()

다른 장비의 데이터를 불러오기 위해서는, 아래와 같은 함수를 사용하실 수 있습니다. 

In [ ]:
df_0=pd.read_csv('/mnt/elice/dataset/mySuni_PJT_1_Data_bler.csv')
df_1=pd.read_csv('/mnt/elice/dataset/mySuni_PJT_1_Data_connection.csv')
df_2=pd.read_csv('/mnt/elice/dataset/mySuni_PJT_1_Data_interx2in_succ_rate.csv')
df_3=pd.read_csv('/mnt/elice/dataset/mySuni_PJT_1_Data_interx2out_succ_rate.csv')
df_4=pd.read_csv('/mnt/elice/dataset/mySuni_PJT_1_Data_intraenb_succ_rate.csv')
df_5=pd.read_csv('/mnt/elice/dataset/mySuni_PJT_1_Data_PRB.csv')
df_6=pd.read_csv('/mnt/elice/dataset/mySuni_PJT_1_Data_reconfig.csv')

df_list = [df_0,df_1,df_2,df_3,df_4,df_5,df_6]

`preprocess_data` 모듈의 `return_preprocess_data(df_list, machine_name, order)` 함수를 활용하여 결측치가 보간된 데이터를 얻을 수 있습니다. 

machine_name 에는 원하는 장비명을, order에는 원하는 결측치 보간의 polynomial order 값을 입력해주세요.

예를 들어, `1005_1` 장비의 전처리된 데이터를 불러오기 위해서는 아래와 같은 코드를 사용할 수 있습니다. 

In [ ]:
import preprocess_data

df_new = preprocess_data.return_preprocess_data(df_list, '1005_1', 2)

In [ ]:
df_new

장비 명은 다음의 txt 파일에 저장되어있습니다. 

In [ ]:
f = open('./machine_list.txt')
for i in f:
    print(i)

## 2. 데이터 기본 분석하기
---

데이터의 Descriptive Statistics를 확인합니다.

In [ ]:
df.describe()

#### 분포가 정규분포를 따를까?

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=2, figsize=(15,20))

sns.distplot(df["dl_bler_inter"], ax=ax[0,0])
sns.distplot(df["ul_bler_inter"], ax=ax[0,1])
sns.distplot(df["conn_avg_inter"], ax=ax[1,0])
sns.distplot(df["conn_max_inter"], ax=ax[1,1])
sns.distplot(df["interx2in_succ_rate_inter"], ax=ax[2,0])
sns.distplot(df["interx2out_succ_rate_inter"], ax=ax[2,1])
sns.distplot(df["intraenb_succ_rate_inter"], ax=ax[3,0])
sns.distplot(df["dl_prb_inter"], ax=ax[3,1])
sns.distplot(df["ul_prb_inter"], ax=ax[4,0])
sns.distplot(df["reconfig_succ_rate_inter"], ax=ax[4,1])

In [ ]:
corr=df.corr()
corr

변수끼리의 상관성을 확인합니다. 이 때, 검은색에 가까울수록 +상관도가 높고, 흰색에 가까울수록 –상관도가 높습니다.

In [ ]:
sns.heatmap(corr, cmap="OrRd")

seaborn에서 제공하는 pairplot 시각화

In [ ]:
sns.pairplot(df)

## 3. 여러 개의 변수를 통합하여 이상 장비 확인하기
---

#### 두 변수들 사이의 관계 시각화

In [ ]:
scaler_dl_prb = StandardScaler().fit_transform(df.dl_prb_inter.values.reshape(-1,1))
scaler_ul_prb = StandardScaler().fit_transform(df.ul_prb_inter.values.reshape(-1,1))
scaler_dl_bler = StandardScaler().fit_transform(df.dl_bler_inter.values.reshape(-1,1))
scaler_ul_bler = StandardScaler().fit_transform(df.ul_bler_inter.values.reshape(-1,1))
scaler_conn_max = StandardScaler().fit_transform(df.conn_max_inter.values.reshape(-1,1))
scaler_conn_avg = StandardScaler().fit_transform(df.conn_avg_inter.values.reshape(-1,1))

dl_prb & ul_prb 관계

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_dl_prb[0:200], label='dl_prb_inter')
plt.plot(scaler_ul_prb[0:200], label='ul_prb_inter')

plt.legend(loc='upper left')

plt.show()

ul_bler & dl_bler 관계

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_dl_bler[0:200], label='dl_bler_inter')
plt.plot(scaler_ul_bler[0:200], label='ul_bler_inter')

plt.legend(loc='upper left')

plt.show()

conn_max & conn_avg 관계

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_conn_max[0:200], label='conn_max_inter')
plt.plot(scaler_conn_avg[0:200], label='conn_avg_inter')

plt.legend(loc='upper left')

plt.show()

conn_max 와 연결 무선 자원에 대한 지표와의 관계

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_conn_max[0:200], label='conn_max_inter')
plt.plot(scaler_ul_prb[0:200], label='ul_prb_inter')

plt.legend(loc='upper left')

plt.show()

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_conn_max[0:200], label='conn_max_inter')
plt.plot(scaler_dl_prb[0:200], label='dl_prb_inter')

plt.legend(loc='upper left')

plt.show()

conn_avg 와 연결 무선 자원에 대한 지표와의 관계

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_conn_avg[0:200], label='conn_avg_inter')
plt.plot(scaler_ul_prb[0:200], label='ul_prb_inter')

plt.legend(loc='upper left')

plt.show()

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_conn_avg[0:200], label='conn_avg_inter')
plt.plot(scaler_dl_prb[0:200], label='dl_prb_inter')

plt.legend(loc='upper left')

plt.show()

연결 재구성 성공률과 연결 데이터 블록 오류율과의 관계

In [ ]:
scaler_reconfig_succ_rate = StandardScaler().fit_transform(df.reconfig_succ_rate_inter.values.reshape(-1,1))

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_dl_bler[0:200], label='dl_bler_inter')
plt.plot(scaler_reconfig_succ_rate[0:200], label='reconfig_succ_rate_inter')

plt.legend(loc='upper left')

plt.show()

In [ ]:
fig = plt.subplots(figsize=(20,5))
plt.plot(scaler_ul_bler[0:200], label='ul_bler_inter')
plt.plot(scaler_reconfig_succ_rate[0:200], label='reconfig_succ_rate_inter')

plt.legend(loc='upper left')

plt.show()

### [TODO] 1005_1 장비에 대하여 동일하게 적용해봅니다. 

df_new를 활용하여 Descriptive Statistics 를 확인하고, 연결 재구성 성공률과 연결 데이터 블록 오류율과의 관계에 대한 비교 시각화를 진행해봅니다.

In [ ]:
'code'

In [ ]:
scaler_dl_bler_new = 'code'

In [ ]:
scaler_reconfig_succ_rate_new = 'code'

In [ ]:
fig = plt.subplots(figsize=(20,5))

'code'

plt.legend(loc='upper left')

plt.show()

## 4. STL 기법을 활용한 이상치 탐지 
---

STL 기법이란, time series = seasonal + trend 구분하여, random graph 에서 비정상 포인트를 발견하는 방법입니다. STL 기법을 활용하여 이상치를 탐지해봅니다.

In [ ]:
!pip install statsmodels

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
decomposition = sm.tsa.seasonal_decompose(df["dl_prb_inter"][:500], model='additive', period=24)
fig=decomposition.plot()
fig.set_size_inches(10,10)
plt.show()

## 5. Autocorrelation/Partial-Autocorrelation plot(자기상관 그래프)을 통한 이상탐지
---

Autocorrelation/Partial-Autocorrelation plot(자기상관 그래프)를 통해 데이터의 이상치를 탐지합니다.

#### ACF/PACF 기법

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(np.arange(500), df["dl_prb_inter"][:500])

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,5))
fig.suptitle('Raw Data')
sm.graphics.tsa.plot_acf(df["dl_prb_inter"], lags=70, ax=ax[0])
sm.graphics.tsa.plot_pacf(df["dl_prb_inter"], lags=70, ax=ax[1]);

#### ACF/PACF 0~500까지

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,5))
fig.suptitle('Raw Data')
sm.graphics.tsa.plot_acf(df["dl_prb_inter"], lags=500, ax=ax[0])
sm.graphics.tsa.plot_pacf(df["dl_prb_inter"], lags=500, ax=ax[1]);